In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os 

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator


from sklearn.preprocessing import StandardScaler

import logging

logging.basicConfig(level = logging.INFO)


gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)
    
    
logging.info("Tensorflow Version is {}".format(tf.__version__))
logging.info("Keras Version is {}".format(tf.keras.__version__))
from tensorflow.python.client import device_lib
logging.info(device_lib.list_local_devices())
tf.device('/device:XLA_GPU:0')

INFO:root:Tensorflow Version is 2.4.0-rc3
INFO:root:Keras Version is 2.4.0
INFO:root:[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16187930958393823820
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2097152000
locality {
  bus_id: 1
  links {
  }
}
incarnation: 144647930059963153
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:03:00.0, compute capability: 7.5"
]


In [5]:
data = np.load('/home/ML4NO/Data/n1000000_0910_classification.npz')

In [7]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])
target = data['cpv']

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [17]:
def DNN_Model(name):
    model_DNN = Sequential(name = "Model_DNN_"+str(name))

#     model_DNN.add(keras.Input(shape=(264,), name = 'input'))
# #     model_DNN_1.add(Dense(256, activation='relu', name = 'dense_1'))
#     model_DNN.add(Dense(64, activation='relu', name = 'dense_1'))
#     model_DNN.add(Dense(32, activation='relu', name = 'dense_2'))
# #     model_DNN_1.add(Dense(32, activation='relu', name = 'dense_4'))
#     model_DNN.add(Dense(1, activation='relu', name = 'dense_3'))
# #     model_DNN_1.add(ActivityRegularization(l2=0.1, name = 'Regularization'))
#     model_DNN.add(Dropout(0.00001))
#     

    
    model_DNN.add(keras.Input(shape=(304,), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(32, activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(32, activation='relu', name = 'dense_2'))
    model_DNN.add(Dense(256, activation='relu', name = 'dense_3'))
#     model_DNN_1.add(Dense(32, activation='relu', name = 'dense_4'))
    model_DNN.add(Dense(1, activation='relu', name = 'output'))
#     model_DNN_1.add(ActivityRegularization(l2=0.1, name = 'Regularization'))
#     model_DNN.add(Dropout(0.00001))
    
    # model_opt = keras.optimizers.Adadelta()
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss="binary_crossentropy",#keras.losses.binary_crossentropy
                              optimizer=model_opt,
                              metrics=['accuracy'])

    model_DNN.summary()
    return model_DNN

In [18]:
model = DNN_Model("regression")
check_list=[]
csv_logger = CSVLogger("./DNN_training_log.csv")
# checkpoint = ModelCheckpoint(
#                     filepath= "./"+str(model)+"_KFold/DNN_"+str(model)+"_Models_"+str(int(pt_min))+str(int(pt_max))+"/" + str(model) + "_DNN_checkmodel_"+str(model_index)+".h5",
#                     save_best_only=True,
#                     verbose=0)
# earlystopping = EarlyStopping(
#                     monitor="val_loss",
#                     min_delta=0,
#                     patience=20,
#                     verbose=0,
#                     mode="auto",
#                     baseline=None,
#                     restore_best_weights=False,
#                 )

# check_list.append(checkpoint)
check_list.append(csv_logger)
# check_list.append(earlystopping)

Model: "Model_DNN_regression"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BatchNormalization (BatchNor (None, 304)               1216      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                9760      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               8448      
_________________________________________________________________
output (Dense)               (None, 1)                 257       
Total params: 20,737
Trainable params: 20,129
Non-trainable params: 608
_________________________________________________________________


In [20]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 63s 5ms/step - loss: 0.2078 - accuracy: 0.9307 - val_loss: 0.1701 - val_accuracy: 0.9405
Epoch 2/20
12516/12516 [==============================] - 64s 5ms/step - loss: 0.2021 - accuracy: 0.9325 - val_loss: 0.1694 - val_accuracy: 0.9390
Epoch 3/20
12516/12516 [==============================] - 66s 5ms/step - loss: 0.1984 - accuracy: 0.9335 - val_loss: 0.1705 - val_accuracy: 0.9419
Epoch 4/20
12516/12516 [==============================] - 65s 5ms/step - loss: 0.2056 - accuracy: 0.9327 - val_loss: 0.2228 - val_accuracy: 0.9248
Epoch 5/20
12516/12516 [==============================] - 65s 5ms/step - loss: 0.2094 - accuracy: 0.9304 - val_loss: 0.1717 - val_accuracy: 0.9401
Epoch 6/20
12516/12516 [==============================] - 65s 5ms/step - loss: 0.2007 - accuracy: 0.9340 - val_loss: 0.1724 - val_accuracy: 0.9403
Epoch 7/20
12516/12516 [==============================] - 66s 5ms/step - loss: 0.2096 - accuracy: 0.9311 - val_loss: 0

In [ ]:
index = 1
while os.path.isfile('/home/ML4NO/ML/Classification/models/0910_{}.h5'.format(index)):
    index += 1
model.save('/home/ML4NO/ML/Classification/models/0910_{}.h5'.format(index))

In [ ]:
scale_steps = np.logspace(-3, 0, 100)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

In [ ]:
model_index = index
index = 1
path = '/home/ML4NO/ML/Classification/models_furthurTrain/0910_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML/Classification/models_furthurTrain/0910_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(20):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML/Classification/models_PoissonTrain/0910_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))